In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [6]:
import logging

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    datefmt='%Y-%m-%d %H:%M:%S'
)

# Create logger
logger = logging.getLogger('data_exploration')


In [ ]:
def load_datasets():
    """Load all datasets from the datasets folder"""
    logger.info("Loading datasets into pandas dataframes...")
    users_df = pd.read_csv('datasets/users.csv')
    items_df = pd.read_csv('datasets/items.csv')
    events_df = pd.read_csv('datasets/events.csv')

    return users_df, items_df, events_df

users_df, items_df, events_df = load_datasets()
users_df.head(), items_df.head(), events_df.head()

(   user_id country
 0        0     USA
 1        1     USA
 2        2   Italy
 3        3   Japan
 4        4   Italy,
    item_id category    hair   eyes
 0        0    flirt     red   blue
 1        1    dance   brown   blue
 2        2       dj     red  brown
 3        3    flirt  blonde  brown
 4        4       dj     red  brown,
    user_id  item_id   timestamp
 0        0       34  1607590072
 1        0       26  1607590260
 2        0      187  1607590503
 3        0       86  1607591006
 4        0      111  1607591526)

In [4]:
def remove_duplicates(df):
    """Remove duplicates items with the same session keeping the first records by time"""
    df = (df.sort_values(['user_id', 'session_id', 'timestamp', 'item_id'])
          .drop_duplicates(subset=['user_id', 'session_id', 'item_id'], keep='first')
          .reset_index(drop=True))
    return df

def filter_multievent_sessions(df):

    # Count events per session
    session_counts = df.groupby(['user_id', 'session_id']).size().reset_index(name='event_count')

    # Filter out sessions with only one event
    df = df.merge(session_counts, on=['user_id', 'session_id'])
    df = df[df['event_count'] > 1].drop(columns=['event_count'])
    
    return df


def process_sessions(events):
    
    # Convert timestamp to datetime
    events['datetime'] = pd.to_datetime(events['timestamp'], unit='s')
    # Sort by user_id and timestamp
    events = events.sort_values(by=["user_id", "timestamp"])

    # Calculate time difference between consecutive events
    events['hours_since_last'] = events.groupby('user_id')['datetime'].diff().dt.total_seconds() / 3600

    # Identify new sessions (time_diff > 8 hours)
    events['new_session'] = (events['hours_since_last'] >= 8 ).fillna(True)

    # Assign session IDs
    events['session_counter'] = events.groupby('user_id')['new_session'].cumsum()
    events['session_id'] = events['user_id'].astype(str)+'-'+events['session_counter'].astype(str)

    events = remove_duplicates(events)
    events = filter_multievent_sessions(events)

    # Adding bounce flag column
    events['bounce_flag'] = (events.groupby('session_id')['datetime']
                            .transform('max') == events['datetime']).astype(int)

    return events

In [5]:
events_dedup = process_sessions(events_df)
events_dedup

,user_id,item_id,timestamp,datetime,hours_since_last,new_session,session_counter,session_id,bounce_flag
0,0,34,1607590072,2020-12-10 08:47:52,NaN,False,0,0-0,0
1,0,26,1607590260,2020-12-10 08:51:00,0.052222,False,0,0-0,0
2,0,187,1607590503,2020-12-10 08:55:03,0.067500,False,0,0-0,0
3,0,86,1607591006,2020-12-10 09:03:26,0.139722,False,0,0-0,0
4,0,111,1607591526,2020-12-10 09:12:06,0.144444,False,0,0-0,0
...,...,...,...,...,...,...,...,...,...
15103,499,10,1608042115,2020-12-15 14:21:55,0.120278,False,2,499-2,0
15104,499,76,1608042367,2020-12-15 14:26:07,0.070000,False,2,499-2,0
15105,499,99,1608042850,2020-12-15 14:34:10,0.134167,False,2,499-2,0
15106,499,198,1608043346,2020-12-15 14:42:26,0.137778,False,2,499-2,0


### Total Number of sessions:

In [7]:
print("Total number of sessions with multiple events in a session: ",len(events_dedup.groupby('session_id')))

Total number of sessions with multiple events in a session:  1026
